In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
import geoviews as gv
import cartopy
from cartopy import crs as ccrs
from cartopy import feature as cf
hv.notebook_extension()
%output dpi=120

Cartopy and shapely make working with geometries and shapes very simple.  GeoViews provides convenient means of wrapping the various geometry types they provide. In addition to basic Path and Polygons types, which draw geometries from lists of arrays, GeoViews also provides the ``Feature`` and ``Shape`` types, which wrap cartopy Features and shapely geometries respectively.

### Feature

The Feature Element provides a very convenient means of overlaying a set of basic geographic features on top of or behind a plot. The ``cartopy.feature`` module, which we have imported as ``cf``, provides basic geographic features such as coastlines, country borders, and land masses. Here we demonstrate how we can plot these very easily, either in isolation or overlaid:

In [ ]:
%opts Overlay [show_grid=False] Layout [hspace=0.2 sublabel_position=(-.15, .7)]
coasts  = gv.Feature(cf.COASTLINE)
borders = gv.Feature(cf.BORDERS)
land    = gv.Feature(cf.LAND)

land + borders + land*borders

### Shape

The ``gv.Shape`` object wraps around any shapely geometry, allowing finer grained control over each polygon. We can, for example, access the geometries on the ``LAND`` feature and display them individually. Here we will get the geometry corresponding to the Australian continent and display it using shapely's inbuilt SVG repr (not yet a HoloViews plot, just a bare SVG displayed by Jupyter directly):

In [ ]:
land_geoms = list(cf.LAND.geometries())
land_geoms[21]

Instead of letting shapely render it as an SVG, we can now wrap it in the ``gv.Shape`` object and let matplotlib or bokeh render it, alone or with other GeoViews or HoloViews objects:

In [ ]:
%%opts Points (color="black")
australia = gv.Shape(land_geoms[21], crs=ccrs.PlateCarree())

australia * gv.Points([(133.870,-23.700)]) * hv.Text(133.870,-21.5, 'Alice Springs')

We can also iterate over the geometries and wrap them in an NdOverlay of ``gv.Shape`` Elements:

In [ ]:
%%opts NdOverlay [aspect=2]
hv.NdOverlay({i: gv.Shape(s, crs=ccrs.PlateCarree()) for i, s in enumerate(land_geoms)})

This makes it possible to create choropleth maps, where each part of the geometry is assigned a value that will be used to color it. However, constructing a choropleth by combining a bunch of shapes can be a lot of effort and is error prone. For that reason, the Shape Element provides convenience methods to load geometries from a shapefile. Here we load the boundaries of UK electoral districts directly from an existing shapefile:

In [ ]:
shapefile = './assets/boundaries/boundaries.shp'
gv.Shape.from_shapefile(shapefile, crs=ccrs.PlateCarree())

To combine these shapes with some actual data, we have to be able to merge it with a dataset. To do so we can inspect the records the cartopy shapereader loads:

In [ ]:
shapes = cartopy.io.shapereader.Reader(shapefile)
list(shapes.records())[0]

As we can see the record contains a ``MultiPolygon`` together with a standard geographic ``code``, which we can use to match up the geometries with a dataset. To continue we will require a dataset that is also indexed by these codes. For this purpose we load a dataset of the 2016 EU Referendum result in the UK:

In [ ]:
referendum = pd.read_csv('./assets/referendum.csv')
referendum = hv.Dataset(referendum)
referendum.data.head()

The ``from_records`` function optionally also supports merging the records and dataset directly. To merge them, supply the name of the shared attribute on which the merge is based via the ``on`` argument. If the name of attribute in the records and the dimension in the dataset match exactly, you can simply supply it as a string, otherwise supply a dictionary mapping between the attribute and column name. In this case we want to color the choropleth by the `'leaveVoteshare'`, which we define via the `value` argument. By default, the resulting `NdOverlay` of shapes will be indexed by an integer index. To draw the index from values in the dataset instead, you can request one or more indexes using the ``index`` argument. Finally we will declare the coordinate reference system in which this data is stored, which will in most cases be the simple Plate Carree projection.  We can then view the choropleth, with each shape colored by the specified value (the percentage who voted to leave the EU):

In [ ]:
%%opts Shape (cmap='viridis')
gv.Shape.from_records(shapes.records(), referendum, on='code', value='leaveVoteshare',
                     index=['name', 'regionName'], crs=ccrs.PlateCarree())